In [75]:
import os
import sys
import pandas as pd 
import numpy as np
from data_import import *
from phenotype_correlation import *
from genotype_correlation import *
from baseline_mixed_model import *
import statsmodels.api as sm

# import data
geno_df, pheno_df = import_data_svenson()

Samples:
['F326', 'F327', 'F329', 'F330', 'F331', 'F332', 'F333', 'F334', 'F335', 'F336', 'F337', 'F338', 'F339', 'F340', 'F341', 'F342', 'F343', 'F344', 'F345', 'F346', 'F347', 'F348', 'F349', 'F350', 'F351', 'F352', 'F353', 'F354', 'F355', 'F356', 'F357', 'F358', 'F359', 'F360', 'F361', 'F362', 'F363', 'F364', 'F365', 'F366', 'F367', 'F368', 'F369', 'F371', 'F372', 'F373', 'F374', 'F375', 'F376', 'F377', 'F378', 'F379', 'F380', 'F381', 'F382', 'F383', 'F384', 'F385', 'F386', 'F387', 'F388', 'F389', 'F390', 'F391', 'F392', 'F393', 'F394', 'F395', 'F396', 'F397', 'F398', 'F399', 'F400', 'F401', 'F402', 'F403', 'F405', 'F406', 'F407', 'F408', 'F410', 'F411', 'F412', 'F413', 'F414', 'F415', 'F416', 'F417', 'F418', 'F419', 'F420', 'F421', 'F422', 'F423', 'F424', 'F425', 'M326', 'M327', 'M328', 'M329', 'M330', 'M331', 'M332', 'M333', 'M334', 'M335', 'M336', 'M337', 'M339', 'M340', 'M341', 'M342', 'M343', 'M344', 'M345', 'M347', 'M348', 'M349', 'M350', 'M351', 'M352', 'M353', 'M354', 'M355'

/Users/harryyang/Documents/Research/Class/CS229/cm-229/data_import.py:75: RuntimeWarning: Mean of empty slice
  np.nanmean(geno_df_select_transpose, axis=0))/np.nanstd(geno_df_select_transpose, axis=0)
/Users/harryyang/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [79]:
# testing with ghrelin and leptin 
phenotype_2 = "weight1"
phenotype_1 = "length1"
phenotype_3 = "fat_mri"
missing_rate = 0.5
verbose = False
sample_list = list()

In [80]:
corr_mat = calculate_highly_correlated_phenotypes(pheno_df)

print("The correlation between %s and %s is %f" % (phenotype_1, phenotype_2, corr_mat[phenotype_1][phenotype_2]))

# bind phenotype into list to extract
phenotype_list = [phenotype_1, phenotype_2]

# extract the phenotypes 
geno_select, pheno_select = select_phenotype_multiple_phenotypes(geno_df, pheno_df, phenotype_list = phenotype_list, verbose = verbose)

The correlation between length1 and weight1 is 0.819920


In [84]:
for i in range(10):
    print(i)
    # separate training and test dataset 
    geno_tr, pheno_tr, geno_test, pheno_test, test_sample_list = separate_training_test(geno_select, pheno_select, missing_rate = missing_rate, sample_list_select = sample_list)

    # sklearn test 
    from sklearn.linear_model import Ridge 

    clf = Ridge(alpha = 1.0)
    a = clf.fit(y = pheno_tr[phenotype_2], X = geno_tr.transpose())

    # select top N 

    # select those genotypes only for select run
    top_N = 10
    top_N_idx = np.argsort(abs(a.coef_))[-top_N:]

    top_N_values = [a.coef_[i] for i in top_N_idx]
    

    top_N_snps = geno_tr.iloc[top_N_idx].index
    
    if verbose:
        print(top_N_snps)
        print (top_N_values)

    # perform OLS 
    lm = sm.OLS(endog = pheno_tr[phenotype_2], exog = pheno_tr[phenotype_1]).fit()

    if verbose:	
        print("The linear model summary for predicting phenotype %a based on phenotype %a" % (phenotype_2, phenotype_1))
        print(lm.summary())	
        print(lm.params)	

    # prediction for fixed effect
    predictions_fe = lm.predict(pheno_test[phenotype_1])

    # perform ridge regression on the residual (random effect part)
    residuals = pheno_tr[phenotype_2] - lm.predict(pheno_tr[phenotype_1])

    # lm_re = sm.OLS(endog = residuals, exog = geno_tr.loc[top_N_snps].transpose()).fit_regularized(L1_wt = 1.0)
    clf = Ridge(alpha = 1.0)
    a = clf.fit(y = residuals, X = geno_tr.loc[top_N_snps].transpose())

    # remove duplciates
    geno_test_new = geno_test.loc[:,~geno_test.columns.duplicated()]
    geno_test = geno_test_new[pheno_test[phenotype_2].index]

    prediction_re = a.predict(geno_test.loc[top_N_snps].transpose())
    total_prediction = predictions_fe + prediction_re

    new_model_mse = calculate_MSE(total_prediction, pheno_test[phenotype_2])


    # perform ridge regression on the residual (random effect part)
    residuals = pheno_tr[phenotype_2] - lm.predict(pheno_tr[phenotype_1])

    lm_re = sm.OLS(endog = residuals, exog = geno_tr.loc[top_N_snps].transpose()).fit_regularized(L1_wt = 1.0, alpha = 1.0)

    if verbose: 
        print(lm_re.params)

    predictions_re = lm_re.predict(geno_test.loc[top_N_snps].transpose())

    # combine the result from both
    total_prediction = predictions_fe + predictions_re

    old_mse = calculate_MSE(total_prediction, pheno_test[phenotype_2])

    print(predictions_fe, predictions_re)
    print(new_model_mse, old_mse)
    
    print(phenotype_correlation_analysis(geno_df, pheno_df, phenotype_2, phenotype_1,missing_rate = missing_rate, sample_list= test_sample_list))

0
sample
F326    28.266273
F327    27.921562
F333    30.679247
F336    28.955694
F340    31.368668
F343    30.334536
F347    29.300405
F348    28.266273
F350    29.300405
F351    31.023958
F355    30.334536
F359    31.023958
F360    27.576851
F364    29.300405
F369    28.955694
F377    28.955694
F378    30.679247
F379    29.300405
F380    28.266273
F382    29.300405
F383    30.679247
F385    31.023958
F387    30.334536
F388    29.989826
F389    29.989826
F390    30.679247
F391    29.300405
F392    30.679247
F397    31.023958
F398    29.300405
          ...    
M334    30.334536
M335    32.402800
M347    31.023958
M350    30.679247
M352    31.023958
M354    32.747511
M358    30.334536
M359    27.921562
M366    31.713379
M371    30.334536
M376    31.368668
M378    30.334536
M379    31.713379
M380    31.368668
M381    29.989826
M384    29.300405
M385    31.368668
M386    32.058090
M388    31.368668
M390    31.368668
M391    31.023958
M394    32.058090
M399    28.266273
M400    33.781643
M

The correlation between weight1 and length1 is 0.819920
(2.429180054490303, array(['M363', 'F363', 'F381', 'F345', 'M359', 'M358', 'M420', 'M424',
       'M369', 'M381', 'F346', 'F343', 'M410', 'M409', 'F345', 'F374',
       'M335', 'F422', 'M408', 'M334', 'F425', 'F415', 'M340', 'M394',
       'M402', 'M328', 'F346', 'F420', 'M408', 'M382', 'F420', 'M334',
       'M406', 'M365', 'F408', 'M423', 'F354', 'F339', 'M410', 'F350',
       'F344', 'F345', 'M326', 'M380', 'F341', 'F422', 'F361', 'M415',
       'M374', 'F349', 'F371', 'M423', 'F368', 'M373', 'M331', 'F354',
       'F349', 'M357', 'F331', 'M366', 'M367', 'F393', 'M359', 'M379',
       'M396', 'F361', 'M350', 'M415', 'M411', 'F412', 'F420', 'F381',
       'M340', 'M366', 'F373', 'M421', 'F369', 'M339', 'F354', 'M399',
       'M357', 'F327', 'F373', 'F423', 'F347', 'F394', 'F340', 'F368',
       'M408', 'M357', 'F326', 'M395', 'M382'], dtype='<U4'))
3
sample
F326    27.479034
F327    27.143924
F332    26.808814
F334    27.814144


(2.455092480014907, array(['M370', 'M350', 'M382', 'F414', 'F397', 'M340', 'F340', 'F420',
       'M347', 'F336', 'F357', 'F353', 'M327', 'M347', 'F408', 'F339',
       'M345', 'M423', 'M415', 'M361', 'M415', 'M363', 'M341', 'M390',
       'F362', 'F405', 'M355', 'M347', 'F369', 'M360', 'F383', 'F393',
       'F329', 'M365', 'M367', 'F408', 'F375', 'M367', 'M384', 'F343',
       'F422', 'M375', 'F366', 'F384', 'F365', 'M335', 'M360', 'M366',
       'F393', 'F337', 'M398', 'F390', 'F353', 'M408', 'F369', 'F403',
       'M409', 'M329', 'F342', 'F355', 'M357', 'M339', 'F392', 'M374',
       'M409', 'F362', 'M360', 'M387', 'F365', 'M329', 'F367', 'M382',
       'F420', 'M392', 'M400', 'M339', 'F388', 'M379', 'M395', 'M396',
       'F345', 'F341', 'M361', 'M340', 'M379', 'F344', 'M419', 'M397',
       'M332', 'M405', 'F400', 'M385', 'F406'], dtype='<U4'))
6
sample
F327    27.255541
F330    30.283934
F338    29.947446
F340    30.620423
F341    30.283934
F343    29.610958
F344    30.620423
F3

The correlation between weight1 and length1 is 0.819920
(2.264071763640391, array(['M389', 'F387', 'F387', 'M407', 'M373', 'F397', 'F382', 'M356',
       'M417', 'M411', 'M337', 'F359', 'F422', 'F379', 'M405', 'M423',
       'M333', 'M406', 'F417', 'M330', 'F345', 'F364', 'M372', 'M393',
       'F363', 'F396', 'F421', 'F391', 'M329', 'F414', 'F408', 'F375',
       'F339', 'F416', 'M417', 'M360', 'M359', 'F397', 'M362', 'M398',
       'M385', 'F364', 'M335', 'F336', 'F392', 'F343', 'M388', 'F359',
       'M368', 'F358', 'F365', 'F329', 'M372', 'F372', 'M341', 'F334',
       'F408', 'M361', 'M392', 'F390', 'M390', 'M351', 'M358', 'M339',
       'M359', 'M330', 'F408', 'M347', 'M327', 'F391', 'M369', 'F373',
       'M348', 'F360', 'F331', 'M362', 'M393', 'M333', 'F377', 'F410',
       'M380', 'M369', 'M396', 'M422', 'F335', 'M330', 'M345', 'M409',
       'M370', 'F330', 'M415', 'F400', 'M389'], dtype='<U4'))
9
sample
F336    28.820732
F337    29.163836
F339    30.193148
F340    31.222460


ea